In [6]:
import pickle
import os
import sys
import numpy as np
import pandas as pd
import cv2 as cv

import string

import sys

sys.path.append('..')


from kp_package.account_management import start, stop
from kp_package.io import write


image_folder = '../../Images'
data_file_folder = '../../annotator_data'
config_path = '../../annotator_data/config'

cont = True
lock = False
dirty = False


###Addition in python file
while(cont):
    
    # 1.0 Checking Account Configuration and Data Lock
    
    print("Checking Account Configuration...")
    io_ok, lock, config_ok, annotator_id, annotator, config = start()


    if(not(lock)):
        print("Could not Obtain Lock. Close Other operations First. Terminating...")
        break

    if(not(io_ok)):
        print("Account Not configured. Please Obtain Configuration File from Moderator")
        break

    if(not(config_ok)):
        print("Image path not set. Run set_image_path first")
        break
    
    image_folder = config.get('im_pth', '../../../Images')
    cid = config.get('cid',0)
    print("Welcome " + annotator)



#######

    kp_dataset_file = 'KEYPOINTS_DATASET_{}'.format(annotator_id) + '.csv'
    status_file = 'STATUS_{}'.format(annotator_id) + '.csv'
    offered_file = 'offered_for_review_{}_{}'.format(annotator_id,cid) + '.csv'

    #######

    kp_dataset_path = os.path.join(data_file_folder, kp_dataset_file)
    status_path = os.path.join(data_file_folder, status_file)
    offered_path = os.path.join(data_file_folder, offered_file)

    #######

    kp_dataset = pd.read_csv(kp_dataset_path, index_col = False)
    status = pd.read_csv(status_path, index_col = False)


    sent = status[status['sent_for_review'] == True ]
    sent = sent['file_name'].values
    all = status['file_name'].values
    complete = status[status['success'] == True]
    complete = complete['file_name'].values


    for im in all:
         if(( im in sent) or (im not in complete)):
             kp_dataset = kp_dataset[kp_dataset['img_id'] != im]
         else:
             record = status['file_name'] == im
             status.loc[record,'sent_for_review'] = True
             dirty = True

    if(len(kp_dataset) == 0):
        print("No completed Annotations available for offer")
        io_ok = False
        break

    cid += 1

    config['cid'] = cid
    
    stat = write('pickle', config, config_path)
    if(not stat):
        break

    stat = write('pandas', kp_dataset, offered_path, create = True)
    if(not stat):
        break

    stat = write('pandas', status, status_path)
    if(not stat):
        break

    #kp_dataset.to_csv(offered_path, index = False)
    #status.to_csv(status_path, index = False)
    dirty = False
    print("Offered file " + offered_file + " created")
    cont = False
    
if(dirty):
    a = input("Some of the Write operations have failed. Pl close any open files and press enter to try again")
    stat = write('pandas', status, status_path)
    if(not stat):
        print("Failed Again. Please check data integrity. You may try to save backup dataframes to avoid losing data")   

            
        
    stat = write('pandas', kp_dataset, offered_path, create = True)
    if(not stat):
       print("Failed Again. Please check data integrity. You may try to save backup dataframes to avoid losing data")   
    

# 8.0 Release Lock

if(lock):
        stop()
        print("Lock Released")

print("Session Complete")


Checking Account Configuration...
Welcome Dummy1
Offered file offered_for_review_0_2.csv created
Lock Released
Session Complete
